In [ ]:
import logging
from dataloader import MRI2D


In [ ]:
checkpoint_ebm128 = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/models/latentEBM128_UN_767.pth'
latentebm128_image = get_last_recon(checkpoint_ebm128, 'LatentEBM128')

checkpoint_ebm = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/models/latentEBM_old_UN768.pth'
latentebm_image = get_last_recon(checkpoint_ebm, 'LatentEBM')


def get_last_recon(checkpoint_path, model_type):
    config_path = '/Users/carlahugod/Desktop/UNI/6sem/bach/energy-based-representation-learning/src/config/2DMRI_config.yml'
    dataset_type = 'MRI2D'

    run_name = checkpoint_path.split("/")[-1].split(".")[0]
    logging.info(f"Run name: {run_name}")
    # Load configuration
    config = load_config(config_path)
    logging.info(f"Loaded config: {config}")

    run_name = checkpoint_path.split("/")[-1].split(".")[0]
    logging.info(f"Run name: {run_name}")

    dataset = MRI2D(config)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load model
    if model_type == "LatentEBM":
        model_class = LatentEBM
    elif model_type == "LatentEBM128":
        model_class = LatentEBM128
    else:
        raise ValueError(f"Unsupported model type: {model_type}")

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    state_dicts = checkpoint if isinstance(checkpoint, list) else [checkpoint]
    models = [model_class(config, dataset_type).to(device) for _ in range(len(state_dicts))]
    for i, model in enumerate(models):
        model.load_state_dict(state_dicts[i], strict=False)
        model.eval()  # Set to evaluation mode


    # Prepare data
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=config.data_workers)
    im, _ = next(iter(dataloader))
    im = im.to(device)

    # Embed latent
    latent = models[0].embed_latent(im)
    latents = torch.chunk(latent, config.components, dim=1)
    # each latent is a component, and has size (batch_size, latent_dim)

    im_neg = torch.rand_like(im)
    im_negs = gen_image_our(latents, config, models, im_neg, 30)

    our_final_image = im_negs[-1][0].detach().cpu().numpy().transpose(1, 2, 0)
    print(our_final_image.shape)

    return our_final_image

